# Exporting Flowsheets to the Graphical User Interface

# Overview

This tutorial will demonstrate how to export the Filtration RO w/ERD flowsheet to the GUI.

## In order to add a flowsheet to the GUI, click on the "NEW FLOWSHEET +" button in the top right corner of the GUI's flowsheet directory. The first two files MUST be provided, while the latter two are optional:

1.) **Model File**: python file that builds the flowsheet

2.) **Export File**: python file that exports the flowsheet build, solver results, and which variables and parameters should be displayed to the GUI

3.) **Diagram File**: image of the process flow diagram that will be displayed in the GUI (optional)

4.) **Data Files**: yaml, json, csv, or any other necessary data files (optional)

<center><img src="graphics/new_flowsheet_button.png" width="730" height="340"></center>

## Step 1: Create an Export File
We must provide a **Model File** and an **Export File**. For the **Model File**, we will use a copy of the flowsheet we created in the previous tutorial. The remainder of this tutorial will show you how to set up the **Export File**, which contains the following four functions:
<center><img src="graphics/gui_functions.png" width="730" height="340"></center>

## Step 2: Import Modules

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Imports from Pyomo
from pyomo.environ import units as pyunits

# Import FlowsheetInterface, which creates the interface between users, UI developers, and flowsheet models
from idaes_flowsheet_processor.api import FlowsheetInterface

# Import functions from the filtration RO w/ERD flowsheet
from filtration_RO_with_ERD import (
    build,
    scale_system,
    add_costing,
    initialize_system,
    solve_system,
)

## Step 3: Create the export_to_ui Function
Names the GUI flowsheet and calls the other functions in sequential order to create the interface.
<center><img src="graphics/export_to_ui.png" width="730" height="340"></center>

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def export_to_ui():
    return FlowsheetInterface(
        name="Filtration RO with ERD",  # Name of the flowsheet in the GUI
        do_export=export_variables,  # Calls the function to export variables (defined later)
        do_build=build_flowsheet,  # Calls the function to build the flowsheet (defined later)
        do_solve=solve_flowsheet,  # Calls the function to solve the flowsheet (defined later)
    )

## Step 4: Create the do_export Function
Goes unit-by-unit and defines which variables will be displayed, how they will be displayed, and whether they should appear as inputs or outputs
<center><img src="graphics/do_export.png" width="730" height="340"></center>

The following screenshot shows the feed inputs that we'll add to the GUI:
<center><img src="graphics/gui_screenshot.png" width="730" height="340"></center>

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
# Note that the function name matches what was used in export_to_ui
def export_variables(flowsheet=None, exports=None):
    # Refer to filtration_RO_with_ERD.py as fs
    fs = flowsheet

    # --------------------------------------- Feed Operating Conditions ---------------------------------------------------------
    exports.add(
        obj=fs.feed.properties[0].flow_vol_phase["Liq"],
        name="Feed volumetric flow rate",
        ui_units=pyunits.m**3 / pyunits.s,
        display_units="m3/s",
        rounding=4,
        description="Inlet volumetric flow rate",
        is_input=True,
        input_category="Feed",
        is_output=True,
        output_category="Feed",
    )
    exports.add(
        obj=fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"],
        name="Feed NaCl concentration",
        ui_units=pyunits.kg / pyunits.m**3,
        display_units="kg/m3",
        rounding=2,
        description="Feed NaCl concentration",
        is_input=True,
        input_category="Feed",
        is_output=True,
        output_category="Feed",
    )
    exports.add(
        obj=fs.feed.properties[0].conc_mass_phase_comp["Liq", "TSS"],
        name="Feed TSS concentration",
        ui_units=pyunits.kg / pyunits.m**3,
        display_units="kg/m3",
        rounding=2,
        description="Feed TSS concentration",
        is_input=True,
        input_category="Feed",
        is_output=True,
        output_category="Feed",
    )
    exports.add(
        obj=fs.feed.properties[0].pressure,
        name="Feed pressure",
        ui_units=pyunits.Pa,
        display_units="Pa",
        rounding=2,
        description="Feed pressure",
        is_input=True,
        input_category="Feed",
        is_output=True,
        output_category="Feed",
    )
    exports.add(
        obj=fs.feed.properties[0].temperature,
        name="Feed temperature",
        ui_units=pyunits.K,
        display_units="K",
        rounding=2,
        description="Feed temperature",
        is_input=True,
        input_category="Feed",
        is_output=True,
        output_category="Feed",
    )

    # --------------------------------------------- Pump Operating Conditions -----------------------------------------------------------------
    # Note that the names for input_category and output_category have changed
    exports.add(
        obj=fs.pump.efficiency_pump[0],
        name="Pump efficiency",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="Pump efficiency",
        is_input=True,
        input_category="Pump",
        is_output=True,
        output_category="Pump",
    )
    exports.add(
        obj=fs.pump.control_volume.properties_out[0].pressure,
        name="Pump outlet pressure",
        ui_units=pyunits.Pa,
        display_units="Pa",
        rounding=2,
        description="Pump outlet pressure",
        is_input=True,
        input_category="Pump",
        is_output=True,
        output_category="Pump",
    )

    # --------------------------------------- RO Operating Conditions -----------------------------------------------------------
    exports.add(
        obj=fs.RO.A_comp[0, "H2O"],
        name="RO water permeability coefficient",
        ui_units=pyunits.m / pyunits.Pa / pyunits.s,
        display_units="m/Pa/s",
        rounding=2,
        description="RO water permeability coefficient",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.B_comp[0, "TDS"],
        name="RO salt permeability coefficient",
        ui_units=pyunits.m / pyunits.s,
        display_units="m/s",
        rounding=2,
        description="RO salt permeability coefficient",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.recovery_vol_phase[0, "Liq"],
        name="RO volumetric recovery",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="RO volumetric recovery",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.feed_side.channel_height,
        name="RO channel height",
        ui_units=pyunits.m,
        display_units="m",
        rounding=2,
        description="RO feed side channel height",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.feed_side.spacer_porosity,
        name="RO spacer porosity",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="RO feed side spacer porosity",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.permeate.pressure[0],
        name="RO permeate pressure",
        ui_units=pyunits.Pa,
        display_units="Pa",
        rounding=2,
        description="RO permeate pressure",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )
    exports.add(
        obj=fs.RO.area,
        name="RO total membrane area",
        ui_units=pyunits.m**2,
        display_units="m2",
        rounding=2,
        description="RO total membrane area",
        is_input=True,
        input_category="Reverse Osmosis",
        is_output=True,
        output_category="Reverse Osmosis",
    )

    # --------------------------------------- ERD Operating Conditions -----------------------------------------------------------
    exports.add(
        obj=fs.erd.efficiency_pump[0],
        name="ERD pump efficiency",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="Energy recovery device pump efficiency",
        is_input=True,
        input_category="Energy Recovery Device",
        is_output=True,
        output_category="Energy Recovery Device",
    )
    exports.add(
        obj=fs.erd.control_volume.properties_out[0].pressure,
        name="ERD outlet pressure",
        ui_units=pyunits.Pa,
        display_units="Pa",
        rounding=2,
        description="ERD outlet pressure",
        is_input=True,
        input_category="Energy Recovery Device",
        is_output=True,
        output_category="Energy Recovery Device",
    )

    # ------------------------------------------------ System Costing -------------------------------------------------------------
    # If we don't want to display variables in the outputs, set is_output to False and skip the output_category argument
    exports.add(
        obj=fs.costing.utilization_factor,
        name="Utilization factor",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="Utilization factor - [annual use hours/total hours in year]",
        is_input=True,
        input_category="System costing",
        is_output=False,
    )
    exports.add(
        obj=fs.costing.TIC,
        name="Practical investment factor",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=1,
        description="Practical investment factor - [total investment cost/direct "
        "capital costs]",
        is_input=True,
        input_category="System costing",
        is_output=False,
    )
    exports.add(
        obj=fs.costing.plant_lifetime,
        name="Plant lifetime",
        ui_units=pyunits.year,
        display_units="years",
        rounding=1,
        description="Plant lifetime",
        is_input=True,
        input_category="System costing",
        is_output=False,
    )
    exports.add(
        obj=fs.costing.wacc,
        name="Discount rate",
        ui_units=pyunits.dimensionless,
        display_units="fraction",
        rounding=2,
        description="Discount rate used in calculating the capital annualization",
        is_input=True,
        input_category="System costing",
        is_output=False,
    )
    exports.add(
        obj=fs.costing.electricity_cost,
        name="Electricity cost",
        ui_units=fs.costing.base_currency / pyunits.kWh,
        display_units="$/kWh",
        rounding=3,
        description="Electricity cost",
        is_input=True,
        input_category="System costing",
        is_output=False,
    )

    # ----------------------------------------------- Cost Metrics ------------------------------------------------------------
    # If there are variables that are calculated and/or don't make sense as inputs, set is_input to False and ignore the input_category argument
    exports.add(
        obj=fs.costing.LCOW,
        name="Levelized cost of water",
        ui_units=fs.costing.base_currency / pyunits.m**3,
        display_units="$/m3",
        rounding=3,
        description="Levelized cost of water with respect to product water",
        is_input=False,
        is_output=True,
        output_category="Cost metrics",
    )
    exports.add(
        obj=fs.costing.total_operating_cost,
        name="Total operating cost",
        ui_units=fs.costing.base_currency / pyunits.yr,
        display_units="$/yr",
        rounding=3,
        description="Total operating cost",
        is_input=False,
        is_output=True,
        output_category="Cost metrics",
    )
    exports.add(
        obj=fs.costing.total_capital_cost,
        name="Total capital cost",
        ui_units=fs.costing.base_currency,
        display_units="$",
        rounding=3,
        description="Total capital cost",
        is_input=False,
        is_output=True,
        output_category="Cost metrics",
    )
    exports.add(
        obj=fs.costing.total_annualized_cost,
        name="Total annualized cost",
        ui_units=fs.costing.base_currency / pyunits.yr,
        display_units="$/yr",
        rounding=3,
        description="Total annualized cost",
        is_input=False,
        is_output=True,
        output_category="Cost metrics",
    )

    # ------------------------------------------------ Capital Costs ------------------------------------------------------------
    exports.add(
        obj=fs.filtration.costing.capital_cost,
        name="Filtration capital cost",
        ui_units=fs.costing.base_currency,
        display_units="$",
        rounding=3,
        description="Capital cost of filtration",
        is_input=False,
        is_output=True,
        output_category="Capital costs",
    )
    exports.add(
        obj=fs.pump.costing.capital_cost,
        name="Pump capital cost",
        ui_units=fs.costing.base_currency,
        display_units="$",
        rounding=3,
        description="Capital cost of pump",
        is_input=False,
        is_output=True,
        output_category="Capital costs",
    )
    exports.add(
        obj=fs.RO.costing.capital_cost,
        name="Reverse osmosis capital cost",
        ui_units=fs.costing.base_currency,
        display_units="$",
        rounding=3,
        description="Capital cost of reverse osmosis",
        is_input=False,
        is_output=True,
        output_category="Capital costs",
    )
    exports.add(
        obj=fs.erd.costing.capital_cost,
        name="ERD capital cost",
        ui_units=fs.costing.base_currency,
        display_units="$",
        rounding=3,
        description="Capital cost of energy recovery device",
        is_input=False,
        is_output=True,
        output_category="Capital costs",
    )

    # ---------------------------------------------------- Product -----------------------------------------------------------------
    exports.add(
        obj=fs.product.properties[0].flow_vol_phase["Liq"],
        name="Product flow rate",
        ui_units=pyunits.m**3 / pyunits.s,
        display_units="m3/day",
        rounding=2,
        description="Product stream flow rate",
        is_input=False,
        is_output=True,
        output_category="Product",
    )
    exports.add(
        obj=fs.product.properties[0].flow_mass_phase_comp["Liq", "H2O"],
        name="Product water mass flow rate",
        ui_units=pyunits.kg / pyunits.s,
        display_units="kg/s",
        rounding=2,
        description="Product water mass flow rate",
        is_input=False,
        is_output=True,
        output_category="Product",
    )
    exports.add(
        obj=fs.product.properties[0].flow_mass_phase_comp["Liq", "TDS"],
        name="Product TDS mass flow rate",
        ui_units=pyunits.kg / pyunits.s,
        display_units="kg/s",
        rounding=2,
        description="Product TDS mass flow rate",
        is_input=False,
        is_output=True,
        output_category="Product",
    )

    # ---------------------------------------------- Performance Metrics -----------------------------------------------------------
    # If there's a value we want to report that isn't computed in the flowsheet, we can add custom relationships here
    recovery_vol = (
        fs.product.properties[0].flow_vol_phase["Liq"]
        / fs.feed.properties[0].flow_vol_phase["Liq"]
    )
    exports.add(
        obj=recovery_vol,
        name="Volumetric recovery",
        ui_units=pyunits.dimensionless,
        display_units="m3 of product/m3 of feed",
        rounding=5,
        description="Normalized volumetric recovery",
        is_input=False,
        is_output=True,
        output_category="Normalized performance metrics",
    )

## Try It Yourself
**Task**: Write the code that would add filtration water recovery, NaCl removal fraction, and TSS removal fraction as inputs and outputs to the GUI. Note that running this cell will not work.

<details>
  <summary>Click the arrow for hint #1!</summary>
    
The variable name for water recovery would be defined as: `obj=fs.filtration.recovery_mass_phase_comp["Liq", "H2O"],`
</details>

<details>
  <summary>Click the arrow for hint #2!</summary>
    
The variable names for removal fraction would be defined as follows:
`obj=fs.filtration.removal_fraction_mass_phase_comp["Liq", "NaCl"],`
`obj=fs.filtration.removal_fraction_mass_phase_comp["Liq", "TSS"],`
</details>

In [ ]:
# Add the export for filtration water recovery

# Add the export for filtration NaCl removal fraction

# Add the export for filtration TSS removal fraction

## Step 5: Create the do_build Function
Calls the functions from the flowsheet to build and initialize the flowsheet model
<center><img src="graphics/do_build.png" width="730" height="340"></center>

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def build_flowsheet():
    # Create unit & property models, set operating conditions, and establish Arcs
    m = build()
    # Provides scaling factors for variables
    scale_system(m)
    # Adds costing for the unit models and computes system-level metrics
    add_costing(m)
    # Sequentially initializes the flowsheet from start to finish
    initialize_system(m)  

    return m

## Step 6: Create the do_solve Function
Calls the function from the flowsheet to solve the system
<center><img src="graphics/gui_functions.png" width="730" height="340"></center>

<div style=" padding: 15px; border: 1px solid #ccc; font-family: monospace; border-radius: 4px;">

```python
def solve_flowsheet(flowsheet=None):
    fs = flowsheet
    results = solve_system(fs)
    return results

## Step 7: Upload the files to the GUI
Now we'll do a live demonstration of adding the Filtration RO w/ERD flowsheet to the GUI. If you haven't installed the GUI, visit the documentation page for the IDAES Flowsheet Processor and install the latest version of the software for your system: https://prommis.github.io/idaes-flowsheet-processor-ui/docs/Download/watertap.